In [1]:
from transformers import AutoTokenizer
from optimum.onnxruntime import (
    ORTModelForSeq2SeqLM,
)

model_id = "SEBIS/code_trans_t5_base_commit_generation_transfer_learning_finetune"

# model = ORTModelForSeq2SeqLM.from_pretrained(model_id, export=True)
# print(model)

/home/kevin/commit-t5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model.save_pretrained("ct-base-commits")
model = ORTModelForSeq2SeqLM.from_pretrained("ct-base-commits")

OSError: ct-base-commits is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
from optimum.onnxruntime import  OptimizationConfig, ORTOptimizer, ORTModelForSeq2SeqLM

save_dir = "ct-base-commits-optimized"

# Create the optimizer
optimizer = ORTOptimizer.from_pretrained(model)

# Define the optimization strategy by creating the appropriate configuration
optimization_config = OptimizationConfig(
    optimization_level=2,
    enable_transformers_specific_optimizations=True,
    optimize_for_gpu=False,
)

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
optimized_model = ORTModelForSeq2SeqLM.from_pretrained(save_dir)
tokens = tokenizer("This is a sample input", return_tensors="pt")
outputs = optimized_model.generate(**tokens)

In [ ]:
from optimum.onnxruntime import (
    AutoOptimizationConfig, ORTOptimizer
)

save_dir = "ct-base-commits-onnx"

optimizer = ORTOptimizer.from_pretrained(model)
optimization_config = AutoOptimizationConfig.O3()

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

In [29]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSeq2SeqLM
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# load Seq2Seq model and set model file directory
model_dir = model_onnx.model_save_dir
encoder_quantizer = ORTQuantizer.from_pretrained(model_dir, file_name="encoder_model.onnx")

decoder_quantizer = ORTQuantizer.from_pretrained(model_dir, file_name="decoder_model.onnx")

decoder_wp_quantizer = ORTQuantizer.from_pretrained(model_dir, file_name="decoder_with_past_model.onnx")

quantizer = [encoder_quantizer, decoder_quantizer, decoder_wp_quantizer]
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

for q in quantizer:
    q.quantize(save_dir="../models/ct-base-commits-quantized", quantization_config=dqconfig)  # doctest: +IGNORE_RESULT


Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...


In [9]:
model_onnx = ORTModelForSeq2SeqLM.from_pretrained("../models/ct-base-commits-onnx")

Generation config file not found, using a generation config created from the model config.


In [ ]:
model_onnx.push_to_hub("../models/ct-base-commits-onnx.tmp", repository_id="ct-base-commits-onnx")

In [2]:
model_quantized = ORTModelForSeq2SeqLM.from_pretrained("../models/ct-base-commits-quantized")

Generation config file not found, using a generation config created from the model config.


In [3]:
%%time

from optimum.pipelines import pipeline

onnx_seq2seq = pipeline("text2text-generation", model=model_quantized, accelerator="ort")
input_text = """
diff --git a/README.md b/README.md
index 9b077d6..c9ee84c 100644
--- a/README.md
+++ b/README.md
@@ -6,7 +6,7 @@ React is a JavaScript library for building user interfaces.
 * **Efficient:** React minimizes interactions with the DOM by using a mock representation of the DOM.
 * **Flexible:** React works with the libraries and frameworks that you already know.
 
-[Learn how to use React in your own project.](http://facebook.github.io/docs/getting-started.html)
+[Learn how to use React in your own project.](http://facebook.github.io/react/docs/getting-started.html)
 
 ## Examples
 
@@ -41,7 +41,7 @@ The fastest way to get started is to serve JavaScript from the CDN:
 <script src="http://fb.me/JSXTransformer-0.3.0.js"></script>
 ```
 
-We've also built a [starter kit](#) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
+We've also built a [starter kit](http://facebook.github.io/react/downloads/react-0.3.0.zip) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
 
 If you'd like to use [bower](http://bower.io), it's as easy as:
"""
onnx_seq2seq(input_text)

CPU times: user 4.85 s, sys: 4.68 ms, total: 4.85 s
Wall time: 609 ms


[{'generated_text': 'Fixing documentationLink incorrect twice'}]

In [ ]:
model = ORTModelForSeq2SeqLM.from_pretrained(
    "kevinlu1248/ct-base-commits-onnx", export=True
)